# Convert an existing Relational data model into a Dimensional data model

A data warehouse is almost exclusively built in a form of a dimensional data model (or Star schema). The goal here is to :
>- facilitate readibility of the database by non-technical people.
>- facilate queries performance with less joins to be performed between tables.

Dataset here: https://www.postgresqltutorial.com/postgresql-getting-started/postgresql-sample-database/

Note: Initial DB is a relational database respecting the Normalisation (3NF) rules. This DB contains the following tables:
>- Actor
>- addrerss
>- category
>- city
>- country
>- customer
>- film
>- film_actor
>- film_category
>- inventory
>- language
>- payment
>- rental
>- staff
>- store

### Connection to the DB

In [1]:
import psycopg2

def connection_to_db():
    conn = psycopg2.connect(
        database = "dvdrental",
        user = "postgres",
        host = "localhost",
        port = 5432,
        password = "@SSitanecl94"
    )
    cur = conn.cursor()
    print("Connection to the database successfull!")

    return cur, conn


### Create the new DB for the data warehouse (Star schema)

Here, we are converting the initial relational DB into a dimensional (start schema) for the data warehouse.

We have aggregated all the table in the initial relational DB into the following:
>- (fact table) **factsales**
>- (Dimension) **dimdate**
>- (Dimension) **dimcustomers**
>- (Dimension) **dimstore**
>- (Dimension) **dimmovie**

In [2]:
create_movie = """
    CREATE TABLE dimmovie (
        movie_key SERIAL NOT NULL PRIMARY KEY,
        film_id INT NOT NULL,
        title VARCHAR(255) NOT NULL,
        description TEXT,
        release_year YEAR NOT NULL,
        language VARCHAR(20) NOT NULL,
        rental_duration VARCHAR(20) NOT NULL,
        original_language VARCHAR(20) NOT NULL,
        length INT NOT NULL,
        ratings VARCHAR(5) NOT NULL,
        special_features VARCHAR(60) NOT NULL
    );
"""

crete_dimdate = """
    CREATE TABLE dimdate (
        date_key INT NOT NULL PRIMARY KEY,
        date DATE NOT NULL,
        year INT NOT NULL,
        quarter INT NOT NULL,
        month INT NOT NULL,
        week INT NOT NULL,
        day INT NOT NULL,
        is_weekend BOOLEAN NOT NULL
    );
"""

create_dimstore = """
    CREATE TABLE dimstore (
        store_key INT NOT NULL PRIMARY KEY,
        store_id INT NOT NULL,
        address VARCHAR(50) NOT NULL,
        address2 VARCHAR(50) NOT NULL,
        district VARCHAR(20) NOT NULL,
        city VARCHAR(50) NOT NULL,
        country VARCHAR(50) NOT NULL,
        postal_code VARCHAR(10) NOT NULL,
        manager_first_name VARCHAR(50) NOT NULL,
        manager_last_name VARCHAR(50) NOT NULL,
        start_date DATE NOT NULL,
        end_date DATE NOT NULL
    );
"""

create_dimcustomer = """
    CREATE TABLE dimcustomer (
        customer_key SERIAL NOT NULL PRIMARY KEY,
        customer_id INT NOT NULL,
        first_name VARCHAR(50) NOT NULL,
        last_name VARCHAR(50) NOT NULL,
        email VARCHAR(50) NOT NULL,
        address VARCHAR(50) NOT NULL,
        address2 VARCHAR(50) NOT NULL,
        district VARCHAR(20) NOT NULL,
        city VARCHAR(50) NOT NULL,
        country VARCHAR(50) NOT NULL,
        postal_code VARCHAR(10) NOT NULL,
        create_date DATE NOT NULL,
        start_date DATE NOT NULL,
        end_date DATE NOT NULL
    );
"""

create_factsales = """
    CREATE TABLE factsales (
        fact_key SERIAL NOT NULL PRIMARY KEY,
        sales_key INT NOT NULL,
        date_key INT NOT NULL,
        customer_key INT NOT NULL,
        movie_key INT NOT NULL,
        store_key INT NOT NULL,
        sales_amoount INT NOT NULL,
        FOREIGN KEY (movie_key) REFERENCES dimmovie (movie_key),
        FOREIGN KEY (date_key) REFERENCES dimdate (date_key),
        FOREIGN KEY (store_key) REFERENCES dimstore (store_key),
        FOREIGN KEY (customer_key) REFERENCES dimcustomer (customer_key)
    ) ;

"""

create_tables_queries = [create_movie, crete_dimdate, create_dimcustomer, create_dimstore, create_factsales]

In [4]:
drop_movie = "DROP TABLE dimmovie"
drop_dimdate = "DROP TABLE dimdate"
drop_dimstore = "DROP TABLE dimstore"
drop_dimcustomer = "DROP TABLE dimcustomer"
drop_factsales = "DROP TABLE factsales"

drop_tables_queries = [drop_movie, drop_dimdate, drop_dimstore, drop_dimcustomer, drop_factsales]

In [5]:
# # Create connection to the DB
cur, conn = connection_to_db()

Connection to the database successfull!


In [6]:
# Drop tables if they existent
for query in drop_tables_queries:
    cur.execute(query)
    conn.commit()

# Create tables if they existent
for query in create_tables_queries:
    cur.execute(query)
    conn.commit()

### Insert values into our dimensional DB tables in the data warehouse

In [7]:
insert_movie = """
    INSERT INTO dimmovie (film_id, title, description, release_year, language, rental_duration,
        original_language, length, ratings, special_features)

        SELECT 
            f.film_id,
            f.title,
            f.description,
            f.release_year,
            l.name,
            f.rental_duration,
            --> original_language,
            f.length,
            f.rating,
            f.special_features
        FROM film as f
            JOIN language as l
                ON f.language_id = l.language_id
    ;
"""

insert_dimdate = """
    INSERT INTO dimdate (date_key, date, year, quarter, month, week, day, is_weekend)
        SELECT 
    
   
   ;
"""

insert_dimstore = """
    CREATE TABLE dimstore (store_key, store_id, address, address2, district, city, country, postal_code,
        manager_first_name, manager_last_name, start_date, end_date)
        
        SELECT
            --> store_key,
            s.store_id,
            a.address,
            a.address2,
            a.district,
            ci.city,
            co.country,
            a.postal_code,
            --> manager_first_name,
            --> manager_last_name,
            --> start_date,
            --> end_date
        FROM store as s
            JOIN address as a
                ON s.address_id = a.address_id
            JOIN city as ci
                ON ci.city_id = a.city_id
            JOIN country as co
                ON co.country_id = ci.country_id
    ;
        ;
"""

insert_dimcustomer = """
    INSERT INTO dimcustomer (customer_id, first_name, last_name, email, address,
        address2, district, city, country, postal_code, create_date, start_date, end_date)
    
        SELECT 
            c.customer_id,
            c.first_name,
            c.last_name,
            c.email,
            a.address,
            a.address2,
            a.district,
            ci.city,
            co.country,
            a.postal_code,
            c.create_date,
            --> start_date,
            --> end_date
        FROM customer as c
            JOIN address as a
                ON c.address_id = a.address_id
            JOIN city as ci
                ON ci.city_id = a.city_id
            JOIN country as co
                ON co.country_id = ci.country_id
    ;
"""

insert_factsales = """
    INSERT INTO factsales (sales_key, date_key, customer_key, movie_key, store_key, sales_amoount)

        SELECT 
            sales_key,
            date_key,
            customer_key,
            movie_key,
            s.store_key,
            sales_amount
        FROM dimmovie as s
    ;
"""

create_tables_queries = [insert_movie, insert_dimdate, insert_dimstore, insert_dimcustomer, insert_factsales]